In [ ]:
# import pandas as pd

# df_m = pd.read_csv('7장/product_plan_material.csv', index_col = '제품')
# df_p = pd.read_csv('7장/product_plan_profit.csv', index_col = '제품')
# df_s = pd.read_csv('7장/product_plan_stock.csv', index_col = '항목')
# df_plan = pd.read_csv('7장/product_plan.csv', index_col = '제품')

# display(df_m, df_p, df_s, df_plan)

In [6]:
import pandas as pd
import numpy as np
from itertools import product
from ortools.linear_solver import pywraplp

df_m = pd.DataFrame({'원료1':[1, 2],
                    '원료2':[4, 4],
                    '원료3':[3, 1]},
                    index = ['제품1', '제품2'])   #index 지정
df_p = pd.DataFrame({'이익':[5, 4]},
                    index = ['제품1', '제품2'])   #index 지정
df_s = pd.DataFrame({'원료1':[40],
                    '원료2':[80],
                    '원료3':[50]},
                    index = ['재고'])   #index 지정
df_plan = pd.DataFrame({'생산량':[16, 0]},
                    index = ['제품1', '제품2'])   #index 지정

In [5]:
def product_plan(df_profit, df_plan):
    profit = 0
    for i in range(len(df_profit.index)):
        profit += df_profit.iloc[i][0] * df_plan.iloc[i][0] #이익 * 생산량 
    return profit
    
print('총이익: ', product_plan(df_p, df_plan))

총이익:  80


In [11]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')
if not solver:
    print('Not Sovler')

infinity = solver.infinity()
v2 = {}
for i in range(len(df_p)):
    v2[i] = solver.IntVar(0, infinity, 'v%i' % i)
print('Number of variables =', solver.NumVariables())

Number of variables = 2


{0: v0, 1: v1}

In [ ]:
# #이제 m2에 제약조건을 하나씩 추가해보자.

# ##각 제품별 이익*생산량 
# m2 += lpSum(df_p.iloc[i] * v2[i] for i in range(len(df_p))) 

# ##제약조건
# for j in range(len(df_m.columns)): #j: 원료 0,1,2 / i: 제품 0, 1
#     m2 += lpSum(df_m.iloc[i, j] * v2[i] for i in range(len(df_p))) <= df_s.iloc[:, j] #제약조건: 각 제품에 필요한 생산량*원료합 <= 최대 원료 재고량

# m2.solve() #최적해 구해줘!

In [21]:
objective = solver.Objective()
for i in range(len(df_p)):
    objective.SetCoefficient(v2[i], int(df_p.iloc[i]))
objective.SetMaximization()

In [22]:
for j in range(len(df_m.columns)): #j: 원료 0,1,2 / i: 제품 0, 1
    constraint = solver.Constraint(0, int(df_s.iloc[:, j]))
    for i in range(len(df_p)):
        constraint.SetCoefficient(v2[i], int(df_m.iloc[i, j]))

In [25]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', int(solver.Objective().Value()))
    for i in range(len(v2)):
        print(v2[i].name(), ' = ', v2[i].solution_value())
    print()
    print('Problem solved in %f milliseconds' % solver.wall_time())
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())
else:
    print('The problem does not have an optimal solution.')

Objective value = 95
v0  =  15.0
v1  =  5.0

Problem solved in 1080859.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
